In [108]:
# using MetadataTools,  DocStringExtensions
using Random: shuffle, MersenneTwister

# export MulticlassPerceptronClassifier, fit!, predict
using LinearAlgebra: mul!
using SparseArrays

import MLJBase
using MLJ
#using Revise

#> needed for classifiers:
using CategoricalArrays

### Implementing the model struct

**A model is an object storing hyperparameters associated with some machine learning algorithm. In MLJ, hyperparameters include configuration parameters, like the number of threads, and special instructions, such as "compute feature rankings", which may or may not affect the final learning outcome. However, the logging level (verbosity below) is excluded.
**


In MLJ I would do

```
mutable struct MulticlassPerceptronClassifier <: MLJBase.Deterministic
    n_epochs::Int
    n_epoch_patience::Int
    f_average_weights::Bool
    f_shuffle_data::Bool
    element_type::DataType
end
```




In [2]:
mutable struct MulticlassPerceptronClassifier <: MLJBase.Deterministic
    n_epochs::Int
    n_epoch_patience::Int
    f_average_weights::Bool
    f_shuffle_data::Bool
    element_type::DataType
end

**
Models (which are mutable) should not be given internal constructors.
**

**
It is recommended that they be given an external lazy keyword constructor of the same name. This constructor defines default values for every field, and optionally corrects invalid field values by calling a clean! method (whose fallback returns an empty message string):
**

In [3]:
# keyword constructor
function MulticlassPerceptronClassifier( ; 
                                        n_epochs=100,
                                        n_epoch_patience=5,
                                        f_average_weights=true,
                                        f_shuffle_data=false,
                                        element_type=Float32)

    model = MulticlassPerceptronClassifier(n_epochs,
                                           n_epoch_patience,
                                           f_average_weights,
                                           f_shuffle_data,
                                           element_type)
    
    message = MLJBase.clean!(model)
    isempty(message) || @warn message
    return model
end

MulticlassPerceptronClassifier

In [4]:
model = MulticlassPerceptronClassifier()

MulticlassPerceptronClassifier(n_epochs = 100,
                               n_epoch_patience = 5,
                               f_average_weights = true,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 3…18

The function MLJBase.clean is used to change the model hyperparameters in case they are set in an invalid way.

In [5]:
function MLJ.clean!(model::MulticlassPerceptronClassifier)
    warning = ""
    if model.n_epochs < 1
        warning *= "Need n_epochs ≥ 1. Resetting n_epochs=100 "
        model.n_epochs = 50
    end
    
    if model.n_epoch_patience <1
        warning *= "Need epoch_patience ≥ 1. Resetting epoch_patience=5 "
        model.epoch_patience = 5
    end

    return warning
end

## Implementing a `fit` method

In [6]:
mutable struct MulticlassPerceptronClassifierParameters{T}
    W::AbstractMatrix{T}
    b::AbstractVector{T}
    n_classes::Int
    n_features::Int
    is_sparse::Bool
end

#MulticlassPerceptronClassifierParameters(T::Type, n_classes::Int, n_features::Int) = MulticlassPerceptronClassifierParameters{T}(rand(T, n_features, n_classes),
#                                                                                       zeros(T, n_classes),
#                                                                                       n_classes,
#                                                                                       n_features,
#                                                                                       is_sparse)

In [7]:

function MulticlassPerceptronClassifierParameters(T::Type, n_classes::Int, n_features::Int, is_sparse::Bool) 
    
    if is_sparse==false
        return MulticlassPerceptronClassifierParameters{T}(rand(T, n_features, n_classes),
                                                                                       zeros(T, n_classes),
                                                                                       n_classes,
                                                                                       n_features,
                                                                                       is_sparse)
    else
        return  MulticlassPerceptronClassifierParameters{T}(sparse(rand(T, n_features, n_classes)),
                                                            spzeros(T, n_classes),
                                                            n_classes,
                                                            n_features,
                                                            is_sparse)     
    end
end



MulticlassPerceptronClassifierParameters

In [8]:
function MLJBase.fit(model::MulticlassPerceptronClassifier,
                     verbosity::Int,   
                     X,
                     y)
    
    #Xmatrix = MLJBase.matrix(X)
    n_classes    = length(unique(y))
    classes_seen = unique(y)
    n_features   = size(train_x,1)  # this assumes data comes in cols
    
    #decode  = MLJBase.decoder(y[1]) # for predict method
    decode =  false

    # Defining the fitpredict object
    is_sparse = issparse(X)
    perceptron = MulticlassPerceptronClassifierParameters(model.element_type, n_classes, n_features, is_sparse);
    
    ### Fitting code starts
    fit!(perceptron, X, y; 
         verbosity=verbosity, 
         n_epochs=model.n_epochs,
         f_average_weights=model.f_average_weights,
         f_shuffle_data=model.f_shuffle_data
        );   
    
    ### Fitting code ends
    cache = nothing
    fitresult = (perceptron, decode)
    report = NamedTuple{}()
    
    #> return package-specific statistics (eg, feature rankings,
    #> internal estimates of generalization error) in `report`, which
    #> should be a named tuple with the same type every call (can have
    #> empty values)
    return fitresult, cache, report
end

## Let us implement the real `fit!` 


Notice that the definition of `MLJBase.fit` simply contains code to end up encapsulating the `fit!` method that will actually change the model `perceptron`. Now we will implement this `fit!` method.

An important thing that is managed inside `MLJBase.fit` is the encoding and decoding of target classes.

In [10]:
"""
Predicts the class for a given input in a `MulticlassPerceptronClassifier`.
The placeholder is used to avoid allocating memory for each matrix-vector multiplication.

- Returns the predicted class.
"""
function predict_with_placeholder(h::MulticlassPerceptronClassifierParameters, x::AbstractVector, class_placeholder::AbstractVector)
    #@fastmath class_placeholder .= At_mul_B!(class_placeholder, h.W, x) .+ h.b
    class_placeholder .= mul!(class_placeholder, transpose(h.W), x)  .+ h.b
    return argmax(class_placeholder)
end


predict_with_placeholder

In [11]:

"""
Compute the accuracy between `y` and `y_hat`.
"""
function accuracy(y::AbstractVector, y_hat::AbstractVector)
    acc = 0.
    @fastmath for k = 1:length(y)
            @inbounds  acc += y[k] == y_hat[k]
    end
    return acc/length(y_hat)
end


accuracy

fit!(::MulticlassPerceptronClassifierParameters{Float32}, ::Array{Float32,2}, ::Array{Int64,1}; verbosity=1, n_epochs=10, f_average_weights=true, f_shuffle_data=false)

In [12]:
"""


    average_flag=true:
        Maintains a running some of the average weight matrix W and bias vector b.

        One possible way to do this would be to update the weights after every example is seen  but this
        would be less efficient.
"""
function fit!(h::MulticlassPerceptronClassifierParameters, X::AbstractArray, y::AbstractVector;
              verbosity=0,
              n_epochs=50,
              learning_rate=1.,
              f_average_weights=false,
              compute_accuracy=true,
              seed=MersenneTwister(1234),
              f_shuffle_data=false)
    
    n_features, n_samples = size(X)
    @assert length(y) == n_samples
    scores = []
    T = eltype(X)
    counter           = 0
    learning_rate     = T(learning_rate)
    class_placeholder = zeros(T, h.n_classes)
    y_preds           = zeros(Int16, n_samples)
    
    data_indices      = Array(1:n_samples)
    max_acc           = zero(T)
    
    if f_average_weights
        W_average =  zeros(T, h.n_features, h.n_classes)
        b_average =  zeros(T, h.n_classes)
    end

    @fastmath for epoch in 1:n_epochs

        n_mistakes = 0
        if f_shuffle_data
            shuffle!(seed, data_indices)
        end
        
        @inbounds for m in data_indices
            x     = view(X, :, m);
            y_hat = predict_with_placeholder(h, x, class_placeholder)
            
            if y[m] != y_hat
                n_mistakes += 1
                ####  wij ← wij − η (yj −tj) · xi
                h.W[:, y[m]]  .= h.W[:, y[m]]  .+ learning_rate .* x
                h.b[y[m]]      = h.b[y[m]]      + learning_rate
                h.W[:, y_hat] .= h.W[:, y_hat] .- learning_rate .* x
                h.b[y_hat]     = h.b[y_hat]     - learning_rate 
                
                if f_average_weights == true
                    counter_learning_rate = counter * learning_rate
                    W_average[:, y[m]]   .= W_average[:, y[m]]  .+ counter_learning_rate .* x
                    b_average[y[m]]       = b_average[y[m]]      + counter_learning_rate
                    W_average[:, y_hat]  .= W_average[:, y_hat] .- counter_learning_rate .* x
                    b_average[y_hat]      = b_average[y_hat]     - counter_learning_rate
                end
            end
            counter +=1
        end

        acc = (n_samples - n_mistakes)/n_samples
        # push!(scores, acc) maybe it would be nice to return an array with monitoring metrics to 
        # allow users to decide if the model has converged
       
        if verbosity ==1
            print("\r\u1b[K")
            print("Epoch: $(epoch) \t Accuracy: $(round(acc; digits=3))")
        elseif verbosity ==2
            println("Epoch: $(epoch) \t Accuracy: $(round(acc; digits=3))")
        end
    end
    
    if f_average_weights == true
        h.W .= h.W  .- W_average./counter 
        h.b .= h.b  .- b_average./counter
    end
    
end

fit!

In [13]:
using MLDatasets

train_x, train_y = MLDatasets.MNIST.traindata();
test_x, test_y   = MLDatasets.MNIST.testdata();
train_x          = Float32.(train_x);
test_x           = Float32.(test_x);
train_y          = train_y .+ 1;
test_y           = test_y .+ 1;
train_y          = Int64.(train_y);
test_y           = Int64.(test_y);
train_x          = reshape(train_x, 784, 60000);
test_x           = reshape(test_x,  784, 10000);

┌ Info: Recompiling stale cache file /Users/davidbuchacaprats/.julia/compiled/v1.1/MLDatasets/9CUQK.ji for MLDatasets [eb30cadb-4394-5ae3-aed4-317e484a6458]
└ @ Base loading.jl:1184


In [20]:
model = MulticlassPerceptronClassifier(n_epochs=50; f_average_weights=true)

MulticlassPerceptronClassifier(n_epochs = 50,
                               n_epoch_patience = 5,
                               f_average_weights = true,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 3…43

In [21]:
model

MulticlassPerceptronClassifier(n_epochs = 50,
                               n_epoch_patience = 5,
                               f_average_weights = true,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 3…43

In [22]:
@time fitresult, _ , _  = MLJBase.fit(model, 1, train_x, train_y)

Epoch: 50 	 Accuracy: 0.897  9.294647 seconds (19.64 M allocations: 4.334 GiB, 6.70% gc time)


((MulticlassPerceptronClassifierParameters{Float32}(Float32[0.0472152 0.158576 … 0.150542 0.713287; 0.318897 0.981663 … 0.174865 0.830911; … ; 0.592991 0.235063 … 0.668016 0.00751925; 0.797794 0.214817 … 0.761805 0.0689144], Float32[-57.869, 35.9231, 19.744, -22.1515, 5.78087, 102.33, -31.9422, 52.9259, -90.8992, -13.8421], 10, 784, false), false), nothing, NamedTuple())

### Creating a predict

In order to predict in MLJ we need 3 things.

- The model (abstract model definition with hyperparameters)
- The fitresult of the model (containing the learned parameters of the model)
- Data


#### Example of predict for an SVMC

The following code is for predicting with a sklearn SVMC

```
function MLJBase.predict(model::SVMC
                         , fitresult
                         , Xnew)

    xnew = MLJBase.matrix(Xnew)
    result, decode = fitresult
    prediction = ScikitLearn.predict(result, xnew)
    return decode(prediction)
end
```



In [23]:
function predict(h::MulticlassPerceptronClassifierParameters, x::AbstractVector, class_placeholder::AbstractVector)
    class_placeholder .= mul!(class_placeholder, transpose(h.W), x)  .+ h.b
    return argmax(class_placeholder)
end

predict (generic function with 1 method)

In [24]:
"""
Function to predict the class for a given input batch.
- Returns the predicted class.
"""
function predict(h::MulticlassPerceptronClassifierParameters, X::AbstractMatrix)
    predictions = zeros(Int64, size(X, 2))
    class_placeholder = zeros(eltype(h.W), h.n_classes)

    @inbounds for m in 1:length(predictions)
        predictions[m] = predict(h, view(X,:,m), class_placeholder)
    end
    
    return predictions
end

predict

In [25]:
function MLJBase.predict(model::MulticlassPerceptronClassifier, fitresult, Xnew)
    xnew = MLJBase.matrix(Xnew)
    result, decode = fitresult
    prediction = predict(result, xnew)
    #decode(prediction)
    return prediction 
end

In [27]:
@time MLJBase.predict(model,fitresult, train_x[:,1:1000]);

  0.010755 seconds (4.01 k allocations: 3.106 MiB, 59.31% gc time)


In [29]:
@time MLJBase.predict(model,fitresult, view(train_x,:,1:1000));

  0.002261 seconds (4.01 k allocations: 117.750 KiB)


## Evaluating the model 

In [30]:
y_tr_preds = MLJBase.predict(model,fitresult,train_x);
y_te_preds = MLJBase.predict(model,fitresult,test_x);

acc_tr = mean(y_tr_preds .== train_y)
acc_te = mean(y_te_preds .== test_y)

println("train accuracy: $acc_tr, test accuracy: $acc_te")

train accuracy: 0.9357166666666666, test accuracy: 0.9264


### Allowing fit(model,X,y) with categorical arrays


Assume that the data is given with a categorical label which can be represented on when loaded as a string.

Since our target values are actually integers we will create the scenario we want to test with the `catname` mapping.

In [31]:
# this cell retwrites targets as strings to show the point 
# of using CategoricalArrays

catname = Dict(1 => "one",
               2 => "two",
               3 => "three",
               4 => "four",
               5 => "five",
               6 => "six",
               7 => "seven",
               8 => "eight",
               9 => "nine",
               10 => "ten");

train_y_labels = [catname[i] for i in train_y];
test_y_labels  = [catname[i] for i in test_y];

Let us imagine that we are given the data in the following format

In [78]:
train_y_labels[1:3]

3-element Array{String,1}:
 "six" 
 "one" 
 "five"

we should take care of rewritting classes as numbers from 1 to N before we fit our model
(our MulticlassPerceptron implementation uses the indices of the classes update the weight vectors assigned to those indices).

We can make our MLJ model do this automatically inside the `fit` method as long as we provide `train_y_labels`  as a CategoricalArray.

In [79]:
train_y_cat = CategoricalArray(train_y_labels);

In [80]:
train_y_cat[1:10]

10-element CategoricalArray{String,1,UInt32}:
 "six"  
 "one"  
 "five" 
 "two"  
 "ten"  
 "three"
 "two"  
 "four" 
 "two"  
 "five" 

A categorical array will contain

- `x.pool` all possible categories found.
- `x.refs` each value of the array encoded as a refenrence to an element in the pool.
- `levels(x)` returns the possible levels (categories) of x.

In [81]:
length(train_y_cat.pool)

10

In [82]:
train_y_cat[1:5]

5-element CategoricalArray{String,1,UInt32}:
 "six" 
 "one" 
 "five"
 "two" 
 "ten" 

In [83]:
train_y_cat.pool

CategoricalPool{String,UInt32}(["eight","five","four","nine","one","seven","six","ten","three","two"])

In [84]:
train_y_cat.refs[1:5]

5-element Array{UInt32,1}:
 0x00000001
 0x00000002
 0x00000003
 0x00000004
 0x00000005

In [85]:
levels(train_y_cat)

10-element Array{String,1}:
 "eight"
 "five" 
 "four" 
 "nine" 
 "one"  
 "seven"
 "six"  
 "ten"  
 "three"
 "two"  

In [87]:
int(train_y_cat[1:10])

10-element Array{UInt32,1}:
 0x00000007
 0x00000005
 0x00000002
 0x0000000a
 0x00000008
 0x00000009
 0x0000000a
 0x00000003
 0x0000000a
 0x00000002

In [88]:
train_y_cat[1:10]

10-element CategoricalArray{String,1,UInt32}:
 "six"  
 "one"  
 "five" 
 "two"  
 "ten"  
 "three"
 "two"  
 "four" 
 "two"  
 "five" 

#### Mapping categories with a `MLJBase.decoder`

If we want to map from numbers back to strings we need a `MLJBase.CategoricalDecoder` dictionary which can be created using `decoder` from MLJBase.

In [92]:
using MLJBase
dec = decoder(train_y_cat[1])

MLJBase.CategoricalDecoder{String,UInt32}(CategoricalPool{String,UInt32}(["eight","five","four","nine","one","seven","six","ten","three","two"]), [9, 3, 7, 10, 2, 8, 1, 5, 6, 4])

In [95]:
dec(MLJ.int(train_y_cat[1:5]))

5-element CategoricalArray{String,1,UInt32}:
 "six" 
 "one" 
 "five"
 "two" 
 "ten" 

In [96]:
dec([1,1,2])

3-element CategoricalArray{String,1,UInt32}:
 "eight"
 "eight"
 "five" 

In [97]:
dec.pool

CategoricalPool{String,UInt32}(["eight","five","four","nine","one","seven","six","ten","three","two"])

In [98]:
dec(train_y[1]), train_y[1]

(CategoricalString{UInt32} "seven", 6)

# Improving `MLJBase.fit` 

Now we have all ingredients to create

`MLJBase.fit(model::MulticlassPerceptronClassifier, verbosity,X,y)`

Which accepts as input `y` as a Categorical array. Then it does the following:

- Takes a single element from the categorical array (which stores all possible class labels) and from this element it creates  a decoding function that, given an integer it returns back a category. 

```julia
    # decoder maps Integer->Category, used in the predict method
    decode  = MLJBase.decoder(y[1]) 
```


- It maps the categorical array to integers:
```julia
   # Encodes CategoricalArray to an Array of integers
   y = Int.(int(y))  
```

In [99]:

function MLJBase.fit(model::MulticlassPerceptronClassifier,
                     verbosity::Int,   
                     X,
                     y)
    
    #Xmatrix = MLJBase.matrix(X)
    n_classes    = length(unique(y))
    classes_seen = unique(y)
    n_features   = size(X,1)  # this assumes data comes in cols

    decode  = MLJBase.decoder(y[1]) # for the predict method
    y = Int.(int(y))                # Encoding categorical target as array of integers

    is_sparse = issparse(X)
    perceptron = MulticlassPerceptronClassifierParameters(model.element_type, n_classes,
                                                          n_features, is_sparse);

    ### Fitting code starts
    fit!(perceptron, X, y; 
         verbosity=verbosity, 
         n_epochs=model.n_epochs,
         f_average_weights=model.f_average_weights,
         f_shuffle_data=model.f_shuffle_data
        ); 
    
    ### Fitting code ends
    cache = nothing
    fitresult = (perceptron, decode)
    report = NamedTuple{}()
    
    #> return package-specific statistics (eg, feature rankings,
    #> internal estimates of generalization error) in `report`, which
    #> should be a named tuple with the same type every call (can have
    #> empty values)
    return fitresult, cache, report
end


In [100]:
model = MulticlassPerceptronClassifier(n_epochs=20)

MulticlassPerceptronClassifier(n_epochs = 20,
                               n_epoch_patience = 5,
                               f_average_weights = true,
                               f_shuffle_data = false,
                               element_type = Float32,) @ 5…63

In [101]:
@time fitresult, _ , _  = MLJBase.fit(model, 1, train_x, train_y_cat)

Epoch: 20 	 Accuracy: 0.895  3.943002 seconds (8.03 M allocations: 1.803 GiB, 8.18% gc time)


((MulticlassPerceptronClassifierParameters{Float32}(Float32[0.710199 0.0529157 … 0.510163 0.294064; 0.202259 0.335671 … 0.2358 0.649653; … ; 0.767949 0.368562 … 0.443363 0.195588; 0.0152253 0.347191 … 0.132136 0.191593], Float32[48.2356, 4.86863, -17.8378, -85.813, -51.9692, -26.1452, 97.9318, -15.823, 18.6719, 27.8803], 10, 784, false), MLJBase.CategoricalDecoder{String,UInt32}(CategoricalPool{String,UInt32}(["eight","five","four","nine","one","seven","six","ten","three","two"]), [9, 3, 7, 10, 2, 8, 1, 5, 6, 4])), nothing, NamedTuple())

In [102]:
dec = fitresult[2]

MLJBase.CategoricalDecoder{String,UInt32}(CategoricalPool{String,UInt32}(["eight","five","four","nine","one","seven","six","ten","three","two"]), [9, 3, 7, 10, 2, 8, 1, 5, 6, 4])

In [103]:
dec(2)

CategoricalString{UInt32} "five"

In [104]:
function MLJBase.predict(model::MulticlassPerceptronClassifier, fitresult, Xnew)
    xnew = MLJBase.matrix(Xnew)
    result, decode = fitresult
    prediction = predict(result, xnew)
    return decode(prediction)
end

In [105]:
MLJBase.predict(model, fitresult, train_x[:,1:4])

4-element CategoricalArray{String,1,UInt32}:
 "six" 
 "one" 
 "five"
 "two" 

Now we can use directly `.predict`  and get an array with categorical values.

In [107]:
train_y_labels = [catname[i] for i in train_y];
test_y_labels  = [catname[i] for i in test_y];

y_tr_cat   = CategoricalArray(train_y_labels);
y_te_cat   = CategoricalArray(test_y_labels);

y_tr_preds = MLJBase.predict(model,fitresult,train_x);
y_te_preds = MLJBase.predict(model,fitresult,test_x);

acc_tr = mean(y_tr_preds .== y_tr_cat)
acc_te = mean(y_te_preds .== y_te_cat)

println("train accuracy: $acc_tr, test accuracy: $acc_te")

train accuracy: 0.93405, test accuracy: 0.9268


Prediction types for deterministic responses.

In the case of Deterministic models, yhat should be an AbstractVector (commonly a plain Vector) with the same element type as the target y passed to the fit method (see above). Any CategoricalValue or CategoricalString appearing in yhat must have the same levels in its pool as was present in the elements of the target y presented in training, even if not all levels appear in the training data or prediction itself. For example, in the univariate target case, this means MLJ.classes(yhat[i]) = MLJ.classes(y[j]) for all admissible i and j. (The method classes is described under Convenience methods below).

Unfortunately, code not written with the preservation of categorical levels in mind poses special problems. To help with this, MLJBase provides three utility methods: int (for converting a CategoricalValue or CategoricalString into an integer, the ordering of these integers being consistent with that of the pool), decoder (for constructing a callable object that decodes the integers back into CategoricalValue/CategoricalString objects), and classes, for extracting the complete pool from a single value. Refer to Convenience methods below for important details.

Note that a decoder created during fit may need to be bundled with fitresult to make it available to predict during re-encoding. So, for example, if the core algorithm being wrapped by fit expects a nominal target yint of type Vector{<:Integer} then a fit method may look something like this:

### Relevant material about Perceptrons


https://www.codingame.com/playgrounds/9487/deep-learning-from-scratch---theory-and-implementation/perceptrons


https://datascienceplus.com/mnist-for-machine-learning-beginners-with-softmax-regression/


https://medium.com/tebs-lab/how-to-classify-mnist-digits-with-different-neural-network-architectures-39c75a0f03e3



https://www.cs.utah.edu/~zhe/pdf/


#### Adding to the package


- Improve performance, how can we select columns from the traning data without generating views?


- Add Mira implementation of a multiclas perceptron.
